<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TPs/tp2/tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ebooklib beautifulsoup4 pandas

In [ ]:
from ebooklib import epub
from bs4 import BeautifulSoup
import pandas as pd
import re
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

## Funciones Auxiliares

In [ ]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def normalize_text_y(t):
    # Convertir a minúsculas y quitar puntuación
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-Záéíóúüñ0-9¿?,.' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def transformar_etiqueta(y):
  puntuacion_iniciales = []
  puntuacion_finales = []
  capitalizaciones = []
  instancia_id_count = 0
  instancia_ids = []
  token_ids = []
  tokens_l = []

  for parrafo in y:
    inicio_pregunta = False
    palabras = parrafo.split()


    for palabra in palabras:
      tokens = tokenizer.tokenize(palabra.lower())

      for i in range(len(tokens)):
        if tokens[i] == "¿":
          inicio_pregunta = True
          continue
        if tokens[i] == "?" or tokens[i] == "." or tokens[i] == ",":
          continue

        instancia_ids.append(instancia_id_count)
        token_ids.append(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens_l.append(tokens[i])

        if inicio_pregunta:
          puntuacion_iniciales.append('"¿"')
          inicio_pregunta = False
        else:
          puntuacion_iniciales.append("")

        if i != len(tokens) - 1:

          if tokens[i+1] == "?":
            puntuacion_finales.append('"?"')
          elif tokens[i+1] == ".":
            puntuacion_finales.append('"."')
          elif tokens[i+1] == ",":
            puntuacion_finales.append('","')
          else:
            puntuacion_finales.append("")

        else:
          puntuacion_finales.append("")

        if palabra.islower():
          capitalizaciones.append(0)
          ultimo_numero = 0
        elif palabra.istitle():
          capitalizaciones.append(1)
          ultimo_numero = 1
        elif palabra.isupper():
          capitalizaciones.append(3)
          ultimo_numero = 3
        else:
          capitalizaciones.append(2)
          ultimo_numero = 2

    instancia_id_count += 1
  etiquetas = np.stack([token_ids, tokens_l, puntuacion_iniciales, puntuacion_finales, capitalizaciones])
  return etiquetas

In [ ]:
def convertir_epub_a_csv(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

In [ ]:
def crearDataSet(libro):

  datos = pd.DataFrame(columns=['instancia_id', "default", "limpio", 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización'])
  datos['instancia_id'] = df.index
  datos['limpio'] = df['parrafo'].apply(normalize_text_X)
  datos['default'] = df['parrafo'].apply(normalize_text_y)
  datos['token_id'] = np.zeros(len(datos))
  datos['token'] = np.zeros(len(datos))
  datos['punt_inicial'] = np.zeros(len(datos))
  datos['punt_final'] = np.zeros(len(datos))
  datos['capitalización'] = np.zeros(len(datos))

  i = 0
  for p in datos['default']:
    etiquetas = transformar_etiqueta([p])
    datos['token_id'][i] = etiquetas[0]
    datos['token'][i] = etiquetas[1]
    datos['punt_inicial'][i] = etiquetas[2]
    datos['punt_final'][i] = etiquetas[3]
    datos['capitalización'][i] = etiquetas[4]
    i += 1
  return datos

## Cargar libros

Busca libro en formato epub de mi github

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'

In [ ]:
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'

In [ ]:
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

##Codigo

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
def obtener_embedding_de_parrafo(texto: str):
  """
  Devuelve el embedding (estático) para el token.
  """
  tokens = tokenizer.tokenize(texto)
  embeddings = []
  for token in tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id is None or token_id == tokenizer.unk_token_id:
      print(f"❌ El token '{token}' no pertenece al vocabulario de multilingual BERT.")
      return None
    embedding_vector = model.embeddings.word_embeddings.weight[token_id]
    embeddings.append(embedding_vector)
    print(f"✅ Token: '{token}' | ID: {token_id}")
    print(f"Embedding shape: {embedding_vector.shape}")
  return embeddings

In [ ]:
dataSet = crearDataSet('libro1.epub')

In [ ]:
dataSet